# 案件
- 【市場調査】Amazon/ヤフオクの市場の調査をお願いします。初心者歓迎/隙間時間にもできます
- https://crowdworks.jp/public/jobs/9151993

# 課題
- ● Amazon 情報を取れなかった商品は画像情報も取らないようにする。<br>
→　ヤフオクの情報を取った時に画像も取れているので、Amazon 情報を取れなかったらヤフオク画像情報を消せればよい。<br>
→　目視で消すんだったら画像を見るよりも「詳細」ビューにしてファイル名で見る方がラク。<br>
→　2023/05/22：画像削除コード追加完了

# 動作検証

## ヤフオク情報取得テスト

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

url = 'https://auctions.yahoo.co.jp/jp/show/rating?userID=k_h_112&role=seller'
driver.get(url)

In [ ]:
# 下記の商品検索を経由せずに指定の商品ページを直接開く
url = 'https://page.auctions.yahoo.co.jp/jp/auction/s1087711376'
driver.get(url)

name = 'スピード メーター 140km /h 60mm トリップメーター 付 汎用品 ブラック'

In [ ]:
# 一覧ページの HTML 構造を取得する
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# 各商品の親要素を取得する
items = soup.select('td[colspan] > table[bgcolor="#ffffff"]')

# 取得結果の格納用リストを準備する
yafuoku_item_list = []

In [ ]:
for item in items:

    detail_url = item.select('a')[0]['href']
    print(detail_url)

    name = item.select('a')[0].text.rstrip()
    print(name)

    eval_date = item.select('td[width="98%"] small')[0].text.splitlines()[1]
    print(eval_date)

In [ ]:
# 詳細ページを開く
driver.get(detail_url)

In [ ]:
# 詳細ページの HTML 構造を取得する
soup_detail = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# 価格
price = soup_detail.select('dd.Price__value')[0].text.strip()
print(price)

In [ ]:
# 最初に表示されている画像を保存する
# ● 商品名に「/」などのファイル名使用不可文字が入っていると画像保存できない
# →　name.replace('/', '／') などとして商品名から該当文字を置き換えるか取り除く。Amazon商品名も同じ処理をする
import shutil
import os
import requests

image_url = soup_detail.select(f'div.ProductImage__inner img[alt*="{name}"]')[0]['src']
print(image_url)

path = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\images'
file_name = name.replace('/', '／').replace(':', '：') + '【ヤフオク】.' + image_url.split('.')[-1]
print(file_name)
file_path = os.path.join(path, file_name)
print(file_path)

response = requests.get(image_url, stream=True)

with open(file_path, 'wb') as file:
    shutil.copyfileobj(response.raw, file)

In [ ]:
# 送料の詳細ポップアップを開く
driver.find_element(By.ID, 'postageDetailCurrent').click()

In [ ]:
# ポップアップの中から送料を取得する　→　送料を取れるようになった！！！
postage = soup_detail.select('div.BidModal__postagePrice')[0].text
print(postage)

In [ ]:
# ポップアップを閉じる
driver.find_element(By.XPATH, '//a[@class="BidModal__hideButton BidModal__hideButton--large js-modal-hide cl-nofollow"]').click()

In [ ]:
# 取得した情報をリストに追加する
yafuoku_item_list.append([name, price, postage, eval_date, detail_url])
sleep(1)

In [ ]:
# 取得結果を保存する
csv_header = ['商品名', '価格', '送料', '評価日時', '商品ページURL']
csv_date = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクデータ_' + csv_date + '.csv'

with open(csv_file_name, 'w', errors = 'ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_header)
    writer.writerows(yafuoku_item_list)

In [ ]:
print(soup.select('a:contains("次の25件")')[0]['href'])

In [ ]:
driver.find_element(By.XPATH, '(//a[contains(text(),"次の25件")])[1]').click()

In [ ]:
# 待ち時間テスト
print('driver.implicitly_wait(5) 開始')
driver.implicitly_wait(5)
print('driver.implicitly_wait(5) 終了')

## Amazon情報取得テスト

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# Amazon のトップページを開く
url = 'https://www.amazon.co.jp/ref=nav_logo'
driver.get(url)

In [ ]:
# 下記の商品検索を経由せずに指定の商品ページを直接開く
url = 'https://www.amazon.co.jp/Honda-%E3%82%AF%E3%83%A9%E3%82%B7%E3%83%83%E3%82%AF-%E3%82%A8%E3%83%B3%E3%83%96%E3%83%AC%E3%83%A0-215mm%C3%9723mm-%E3%83%AA%E3%83%88%E3%83%AB%E3%82%AB%E3%83%96/dp/B00LVCMEH8/ref=sr_1_2?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&crid=20GQXC9YDP57D&keywords=Honda+%E3%82%AF%E3%83%A9%E3%82%B7%E3%83%83%E3%82%AF+%E3%82%A8%E3%83%B3%E3%83%96%E3%83%AC%E3%83%A0+%E3%83%A1%E3%83%83%E3%82%AD+%E7%AD%86%E8%A8%98%E4%BD%93+215mm%C3%9723mm+%E3%83%9B%E3%83%B3%E3%83%80+%E3%83%A2%E3%83%B3%E3%82%AD%E3%83%BC+%E3%82%B4%E3%83%AA%E3%83%A9+%E3%82%A8%E3%82%A4%E3%83%97+%E3%82%B7%E3%83%A3%E3%83%AA%E3%83%BC+%E3%83%80%E3%83%83%E3%82%AF%E3%82%B9+%E3%83%87%E3%82%A3%E3%82%AA+%E3%82%BA%E3%83%BC%E3%83%9E%E3%83%BC+%E3%83%AA%E3%83%88%E3%83%AB%E3%82%AB%E3%83%96&qid=1681913537&sprefix=honda+%E3%82%AF%E3%83%A9%E3%82%B7%E3%83%83%E3%82%AF+%E3%82%A8%E3%83%B3%E3%83%96%E3%83%AC%E3%83%A0+%E3%83%A1%E3%83%83%E3%82%AD+%E7%AD%86%E8%A8%98%E4%BD%93+215mm+23mm+%E3%83%9B%E3%83%B3%E3%83%80+%E3%83%A2%E3%83%B3%E3%82%AD%E3%83%BC+%E3%82%B4%E3%83%AA%E3%83%A9+%E3%82%A8%E3%82%A4%E3%83%97+%E3%82%B7%E3%83%A3%E3%83%AA%E3%83%BC+%E3%83%80%E3%83%83%E3%82%AF%E3%82%B9+%E3%83%87%E3%82%A3%E3%82%AA+%E3%82%BA%E3%83%BC%E3%83%9E%E3%83%BC+%E3%83%AA%E3%83%88%E3%83%AB%E3%82%AB%E3%83%96%2Caps%2C463&sr=8-2'
driver.get(url)

In [ ]:
yafuoku_name = 'ピクチャーレール用 ステンレスワイヤー 吊り下げ金具 シルバー 4本セット (0.5m フック1個)'
# yafuoku_name = 'Sweetwinds ペニスリング コックリング 二重リング 日本人サイズ 柔らかいシリコン 高弾力 男性用オナニーリング アダル'

# 検索窓に商品名を入力して検索する
driver.find_element(By.ID, 'twotabsearchtextbox').send_keys(yafuoku_name)
driver.find_element(By.ID, 'nav-search-submit-button').click()

In [ ]:
# アダルト商品の場合、年齢確認のポップアップ（新ウィンドウ）が表示されるので「はい」をクリックする
# ただしアダルト商品はそもそもヤフオクで取得できていない
try:
    # 商品ページのリンクをクリックする。新ウィンドウで開かれる
    driver.find_element(By.XPATH, f'(//span[contains(text(),"{yafuoku_name}")]/parent::a[@target="_blank"])[1]').click()
    sleep(1)
except:
    # 年齢認証のポップアップで「はい」を選択する
    driver.find_element(By.ID, 'a-autoid-1-announce').click()
    sleep(1)
    # 商品ページのリンクをクリックする。新ウィンドウで開かれる
    driver.find_element(By.XPATH, f'(//span[contains(text(),"{yafuoku_name}")]/parent::a[@target="_blank"])[1]').click()
    sleep(1)

In [ ]:
# 開いているすべてのウィンドウハンドルを取得する（リスト形式）
handle_array = driver.window_handles
# 新ウィンドウに切り替える
driver.switch_to.window(handle_array[-1])
# 新ウィンドウの HTML 構造を取得する
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
amazon_name = soup.select('span#productTitle')[0].text.strip()
print('Amazon商品名:', amazon_name)

try:
    amazon_price = soup.select('div#corePrice_feature_div span.a-price-whole')[0].text
except:
    amazon_price = soup.select('span#snsDetailPagePrice > span#sns-base-price')[0].text.strip().split()[0]
print('Amazon価格:', amazon_price)

try:
    amazon_tax = soup.select('div#corePrice_feature_div span#taxInclusiveMessage')[0].text.strip().replace('税込', '0')
except:
    amazon_tax = soup.select('span#snsDetailPagePrice > span#sns-base-price > span')[0].text.strip().replace('税込', '0')
print('Amazon税金:', amazon_tax)

amazon_postage = soup.select('span[data-csa-c-content-id="DEXUnifiedCXPDM"]')[0].text.lstrip().replace('配送料 ', '').replace('詳細を見る', '')
print('Amazon送料:', amazon_postage)

amazon_delivery_date = soup.select('div#mir-layout-DELIVERY_BLOCK span.a-text-bold')[0].text.replace(' ', "")
print('Amazon着日:', amazon_delivery_date)

amazon_stock = soup.select('div#availability > span')[0].text.strip()
print('Amazon在庫:', amazon_stock)

amazon_url = driver.current_url
print('Amazon商品URL:', amazon_url)

amazon_info_get_date = datetime.datetime.now()
print('Amazon情報取得日:', amazon_info_get_date)

In [ ]:
# 最初に表示されている画像を保存する
# ● 商品名に「/」「:」などのファイル名使用不可文字が入っていると画像保存できない
# →　yafuoku_name.replace('/', '／') などとして商品名から該当文字を置き換えるか取り除く。ヤフオク商品名も同じ処理をする
import shutil
import os
import requests

image_url = soup.select(f'div#imgTagWrapperId > img[alt*="{yafuoku_name}"]')[0]['src']
print('image_url:', image_url)

path = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\images'
file_name = yafuoku_name.replace('/', '／').replace(':', '：') + '【Amazon】.' + image_url.split('.')[-1]
print('file_name:', file_name)
file_path = os.path.join(path, file_name)
print('file_path:', file_path)

response = requests.get(image_url, stream=True)

with open(file_path, 'wb') as file:
    shutil.copyfileobj(response.raw, file)

In [ ]:
# ヤフオク画像削除
import glob
import os

path = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\images'
remove_file = 'ダイヤ(DAIYA) ベーシックパット TR-433【ヤフオク】 - コピー' + '.jpg'
remove_file_path = os.path.join(path, remove_file)
print('remove_file_path:', remove_file_path)
os.remove(remove_file_path)

## 画像比較テスト

In [ ]:
# 【Python】OpenCVとNumPyで２つの画像を比較（完全一致、部分一致の比率）
# https://office54.net/python/module/opencv-numpy-compare

import cv2
import numpy as np

image1 = cv2.imread('office54.png')
image2 = cv2.imread('office54.png')
image3 = cv2.imread('office54-eng.png')

print(np.array_equal(image1, image2))
print(np.array_equal(image1, image3))
# True
# False

# 画像をリサイズする
img_size = (100, 100)
image1 = cv2.resize(image1, img_size)
image2 = cv2.resize(image2, img_size)
image3 = cv2.resize(image3, img_size)

# 画像をヒストグラム化する
image1_hist = cv2.calcHist([image1], [0], None, [256], [0, 256])
image2_hist = cv2.calcHist([image2], [0], None, [256], [0, 256])
image3_hist = cv2.calcHist([image3], [0], None, [256], [0, 256])

# ヒストグラムした画像を比較
print(cv2.compareHist(image1_hist, image2_hist, 0))
print(cv2.compareHist(image1_hist, image3_hist, 0))
# 1.0
# 0.8882466100801328

In [ ]:
# 【Python】画像の類似度比較が可能なImageHashのインストール
# https://self-development.info/%E3%80%90python%E3%80%91%E7%94%BB%E5%83%8F%E3%81%AE%E9%A1%9E%E4%BC%BC%E5%BA%A6%E6%AF%94%E8%BC%83%E3%81%8C%E5%8F%AF%E8%83%BD%E3%81%AAimagehash%E3%81%AE%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC/

from PIL import Image
import imagehash

# Y_hash = imagehash.average_hash(Image.open('フォグランプ_ヤフオク.jpg'))
Y_hash = imagehash.average_hash(Image.open('ベアリング_ヤフオク.jpg'))
print(Y_hash)

# A_hash = imagehash.average_hash(Image.open('フォグランプ_Amazon.jpg'))
A_hash = imagehash.average_hash(Image.open('ベアリング_Amazon.jpg'))
print(A_hash)

print(Y_hash == A_hash)
print(Y_hash - A_hash)

In [ ]:
from PIL import Image
import imagehash
import os

path = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping'

# Y_name = 'ベアリング_ヤフオク.jpg'
Y_name = 'フォグランプ_ヤフオク.jpg'
Y_path = os.path.join(path, Y_name)
Y_hash = imagehash.average_hash(Image.open(Y_path))
print('Y_hash:', Y_hash)

A_name = 'ベアリング_Amazon.jpg'
# A_name = 'フォグランプ_Amazon.jpg'
A_path = os.path.join(path, A_name)
A_hash = imagehash.average_hash(Image.open(A_path))
print('A_hash:', A_hash)

print('hash完全一致度:', Y_hash == A_hash)
print('hash差:', Y_hash - A_hash)

# 完成版
1. ヤフオクから情報を取得する
1. ヤフオクで取得した情報を基に Amazon で検索する

## 1.ヤフオクの情報を取得する

調査対象セラーの URL（SnC23様提供情報）
- https://auctions.yahoo.co.jp/jp/show/rating?userID=cawjz38692&slider=undefined
- https://auctions.yahoo.co.jp/jp/show/rating?userID=k_h_112&role=seller # 商品名完全一致
- https://auctions.yahoo.co.jp/jp/show/rating?userID=mlamb63401&role=seller # 商品名完全一致
- https://auctions.yahoo.co.jp/jp/show/rating?userID=usaginokono&slider=undefined

調査対象セラーの URL（下遠野調査結果）
- https://auctions.yahoo.co.jp/jp/show/rating?userID=wanwanwankonabe # 商品名部分一致 実績多い
- https://auctions.yahoo.co.jp/jp/show/rating?userID=ebnq4452 # 商品名完全一致あり 実績少ない
- https://auctions.yahoo.co.jp/seller/kaz_type_r_0227 # 実績古い
- https://auctions.yahoo.co.jp/jp/show/rating?userID=fsflm19105&role=seller # 商品名完全一致あり やや少ない

In [ ]:
%%time
# 2125件：Wall time: 2h 14min 6s
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import sys
import datetime
import csv
import pandas as pd
import shutil # 画像保存に使う
import os # 画像保存に使う
import requests # 画像保存に使う

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# 調査対象ページ
url = 'https://auctions.yahoo.co.jp/jp/show/rating?userID=usaginokono&slider=undefined'

# 画像保存用フォルダ
path = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\images'

# 取得結果の格納用リストを準備する
yafuoku_item_list = []

# 「次の25件」のクリックを指定回数繰り返す
for i in range(1):
    driver.get(url)
    driver.implicitly_wait(5)

    # 一覧ページの HTML 構造を取得する
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 各商品情報の親要素を取得する
    items = soup.select('td[colspan] > table[bgcolor="#ffffff"]')

    # 一覧ページ内の商品情報をすべて取得する
    for index, item in enumerate(items):
        try:
            print(f'{i+1}ページ目-{index+1}')

            # 商品名　一覧ページから取得
            name = item.select('a')[0].text.rstrip()
            print(name)

            # 評価日　一覧ページから取得
            # eval_date = item.select('td[width="98%"] small')[0].text.splitlines()[1]
            eval_date = item.select('td[width="98%"] small')[0].text.splitlines()[1]
            # print(eval_date)

            # 詳細ページへの URL　一覧ページから取得
            detail_url = item.select('a')[0]['href']
            # print(detail_url)

            # 詳細ページを開く
            # ■ アダルトカテゴリを開こうとすると年齢認証（ログイン認証）が走り取得できない
            driver.get(detail_url)

            # 詳細ページの HTML 構造を取得する
            soup_detail = BeautifulSoup(driver.page_source, 'html.parser')

            # 価格　詳細ページから取得
            price = soup_detail.select('dd.Price__value')[0].text.strip()
            # print(price)

            # 入札件数　詳細ページから取得
            bid_num = soup_detail.select('span:contains("件")')[0].text
            # print(bid_num)

            # 最初に表示されている画像　詳細ページから取得
            image_url = soup_detail.select(f'div.ProductImage__inner img[alt*="{name}"]')[0]['src']
            # print(image_url)
            file_name = name.replace('/', '／').replace(':', '：') + '【ヤフオク】.' + image_url.split('.')[-1]
            # print(file_name)
            file_path = os.path.join(path, file_name)
            # print(file_path)
            response = requests.get(image_url, stream=True)
            with open(file_path, 'wb') as file:
                shutil.copyfileobj(response.raw, file)

            # 送料の詳細ポップアップを開く
            driver.find_element(By.ID, 'postageDetailCurrent').click()
            driver.implicitly_wait(5)
            # ポップアップの中から送料を取得する　→　送料を取れるようになった！！！
            postage = soup_detail.select('div.BidModal__postagePrice')[0].text
            # print(postage)

            # 送料ポップアップを閉じる
            driver.find_element(By.XPATH, '//a[@class="BidModal__hideButton BidModal__hideButton--large js-modal-hide cl-nofollow"]').click()
            driver.implicitly_wait(5)

            # 取得した情報をリストに追加する
            yafuoku_item_list.append([name, price, postage, bid_num, eval_date, detail_url, image_url])
            # print('● 取得に成功しました ●')
        except:
            # 取得した情報をリストに追加する
            yafuoku_item_list.append([name, '【取得不可】', '【取得不可】', '【取得不可】', eval_date, detail_url, image_url])
            print('◆ 取得に失敗しました ◆')

    # 次のページがある場合は開く。ない場合は処理を終了する
    try:
        url = soup.select('a:contains("次の25件")')[0]['href']
        driver.implicitly_wait(5)
    except:
        driver.quit()
        break

# 取得結果を保存する
csv_header = ['商品名', '価格', '送料', '入札件数', '評価日', '商品ページURL', '画像URL']
csv_date = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクデータ_' + csv_date + '.csv'

with open(csv_file_name, 'w', errors = 'ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_header)
    writer.writerows(yafuoku_item_list)

print('----- ヤフオク情報取得完了 -----')

## 2.ヤフオクの取得データを基に Amazon の情報を検索する

■ 詳細ページのURLをクリックするのではなく、Selenium(?)を使って詳細ページのURLを開く方法にすれば別ウィンドウとしては開かれない。

In [ ]:
%%time
# 283件：Wall time: 27min 54s # sleep(1)
# 376件：Wall time: 49min 25s # sleep(2) に延長
# 500件：Wall time: 1h 10min 32s
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd
import shutil # 画像保存に使う
import os # 画像保存に使う
import requests # 画像保存に使う
import glob # ヤフオク画像削除に使う
from PIL import Image # 画像比較に使う
import imagehash # 画像比較に使う

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# Amazon のトップページを開く
url = 'https://www.amazon.co.jp/ref=nav_logo'
driver.get(url)
driver.implicitly_wait(5)

# 大元のウィンドウの保持
original_window = driver.current_window_handle

# ヤフオクデータ csv の取得
df = pd.read_csv('ヤフオクデータ_20230523012737.csv', encoding='cp932')

# 画像保存用フォルダ
path = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\images'

# 取得情報格納用のリスト準備
yafuoku_amazon_list = []

for index, yafuoku_item in df.iterrows():
    try:
        print(f'{index+1} / {len(df)}')

        # 取得済のヤフオク関連情報を取得する
        yafuoku_name = yafuoku_item['商品名']
        print('ヤフオク商品名:', yafuoku_name)

        yafuoku_price = yafuoku_item['価格'].replace('円（税込 ', '／税込').replace('円（税 ', '／').replace(' 円）', '').split('／')[0]
        # print('ヤフオク価格:', yafuoku_price)
        
        yafuoku_tax = yafuoku_item['価格'].replace('円（税込 ', '／税込').replace('円（税 ', '／').replace(' 円）', '').split('／')[1]
        # print('ヤフオク税金:', yafuoku_tax)

        yafuoku_postage = yafuoku_item['送料'].replace('円（税込）', '')
        # print('ヤフオク送料:', yafuoku_postage)
        
        yafuoku_bid_num = yafuoku_item['入札件数'].replace('件', '')
        # print('ヤフオク入札件数:', yafuoku_bid_num)

        yafuoku_eval_date = yafuoku_item['評価日'].replace('(評価日時：', '').replace(')', '')
        # print('ヤフオク評価日:', yafuoku_eval_date)

        yafuoku_url = yafuoku_item['商品ページURL']
        # print('ヤフオク商品URL:', yafuoku_url)
        
        yafuoku_image_url = yafuoku_item['画像URL']
        # print('ヤフオク商品URL:', yafuoku_image_url)

        # 検索ボックスに商品名を入力する
        driver.find_element(By.ID, 'twotabsearchtextbox').send_keys(yafuoku_name)
        # 検索ボタンをクリックする
        driver.find_element(By.ID, 'nav-search-submit-button').click()
        driver.implicitly_wait(5)

        try:
            # 商品ページのリンクをクリックする。新ウィンドウで開かれる
            driver.find_element(By.XPATH, f'(//span[contains(text(),"{yafuoku_name}")]/parent::a[@target="_blank"])[1]').click()
            driver.implicitly_wait(5)
        except:
            # アダルト商品の場合、年齢確認のポップアップ（新ウィンドウ）が表示されるので「はい」をクリックする
            driver.find_element(By.ID, 'a-autoid-1-announce').click()
            driver.implicitly_wait(5)
            # 商品ページのリンクをクリックする。新ウィンドウで開かれる
            driver.find_element(By.XPATH, f'(//span[contains(text(),"{yafuoku_name}")]/parent::a[@target="_blank"])[1]').click()
            driver.implicitly_wait(5)
            
        # 開いているすべてのウィンドウハンドルを取得する（リスト形式で返ってくる）
        handle_array = driver.window_handles
        # 新ウィンドウに切り替える
        driver.switch_to.window(handle_array[-1])
        # 新ウィンドウの HTML 構造を取得する
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        amazon_name = soup.select('span#productTitle')[0].text.strip()
        # print('Amazon商品名:', amazon_name)

        try:
            amazon_price = soup.select('div#corePrice_feature_div span.a-price-whole')[0].text
        except:
            amazon_price = soup.select('span#snsDetailPagePrice > span#sns-base-price')[0].text.strip().split()[0]
        # print('Amazon価格:', amazon_price)

        try:
            # amazon_tax = soup.select('div#corePrice_feature_div span#taxInclusiveMessage')[0].text.strip()
            amazon_tax = soup.select('div#corePrice_feature_div span#taxInclusiveMessage')[0].text.strip().replace('税込', '0')
        except:
            # amazon_tax = soup.select('span#snsDetailPagePrice > span#sns-base-price > span')[0].text.strip()
            amazon_tax = soup.select('span#snsDetailPagePrice > span#sns-base-price > span')[0].text.strip().replace('税込', '0')
        # print('Amazon税金:', amazon_tax)

        amazon_postage = soup.select('span[data-csa-c-content-id="DEXUnifiedCXPDM"]')[0].text.strip().replace('無料配送', '0／').replace('詳細を見る', '')
        # amazon_postage = soup.select('span[data-csa-c-content-id="DEXUnifiedCXPDM"]')[0].text.strip().replace('詳細を見る', '')
        # print('Amazon送料:', amazon_postage)

        amazon_delivery_date = soup.select('div#mir-layout-DELIVERY_BLOCK span.a-text-bold')[0].text.replace(' ', '')
        # print('Amazon着日:', amazon_delivery_date)

        amazon_stock = soup.select('div#availability > span')[0].text.strip().replace(' ご注文はお早めに', '').replace('。', '')
        # print('Amazon在庫:', amazon_stock)

        amazon_url = driver.current_url
        # print('Amazon商品URL:', amazon_url)

        # 最初に表示されている画像を取得する
        amazon_image_url = soup.select(f'div#imgTagWrapperId > img[alt*="{yafuoku_name}"]')[0]['src']
        # print('Amazon画像URL:', amazon_image_url)
        amazon_image_name = yafuoku_name.replace('/', '／').replace(':', '：') + '【Amazon】.' + amazon_image_url.split('.')[-1]
        # print('Amazon画像ファイル名:', amazon_image_name)
        amazon_image_path = os.path.join(path, amazon_image_name)
        # print('Amazon画像ファイルパス:', amazon_image_path)
        response = requests.get(amazon_image_url, stream=True)
        with open(amazon_image_path, 'wb') as file:
            shutil.copyfileobj(response.raw, file)
            # print('画像を保存しました')
        
        # ヤフオク画像とAmazon画像の一致度を判定する
        yafuoku_image_name = amazon_image_name.replace('【Amazon】', '【ヤフオク】')
        # print('yafuoku_image_name:', yafuoku_image_name)
        # print('amazon_image_name:', amazon_image_name)

        yafuoku_image_path = os.path.join(path, yafuoku_image_name)
        amazon_image_path = os.path.join(path, amazon_image_name)
        
        yafuoku_image_hash = imagehash.average_hash(Image.open(yafuoku_image_path))
        amazon_image_hash = imagehash.average_hash(Image.open(amazon_image_path))
        
        hash_diff = yafuoku_image_hash - amazon_image_hash
        # print('hash_diff:', hash_diff)
        
        # 取得した情報をリストに追加する
        yafuoku_amazon_list.append([
            yafuoku_name, yafuoku_price, yafuoku_tax, yafuoku_postage, yafuoku_bid_num, yafuoku_eval_date, yafuoku_url, yafuoku_image_url, \
            amazon_name, amazon_price, amazon_tax, amazon_postage, amazon_delivery_date, amazon_stock, amazon_url, amazon_image_url, \
            f'=IF(COUNTIF(I{index+2},A{index+2}&"*"),"一致","不一致")', \
            hash_diff, \
            f'=COUNTIF($A$2:A{index+2},A{index+2})', \
            f'=COUNTIF($I$2:I{index+2},I{index+2})', \
            f'=(B{index+2}+D{index+2})*0.9-(J{index+2}+L{index+2})'
        ])

        # 現在のウィンドウを閉じる
        driver.close()
        driver.implicitly_wait(5)

        # ウィンドウハンドラを大元のウィンドウに切り替える
        driver.switch_to.window(original_window)

        # 検索ボックスに残っている文字列を消す
        driver.find_element(By.ID, 'twotabsearchtextbox').clear()
    except:
        # 取得したヤフオク画像を削除する　2023/05/22追加　拡張子の柔軟性はないがひとまず動くようにはできた
        yafuoku_image_name = yafuoku_name.replace('/', '／').replace(':', '：') + '【ヤフオク】.jpg'
        remove_file_path = os.path.join(path, yafuoku_image_name)
        os.remove(remove_file_path)
        print('【Amazon情報を取得できませんでした。画像を削除しました】', yafuoku_image_name)

        yafuoku_amazon_list.append([
            yafuoku_name, yafuoku_price, yafuoku_tax, yafuoku_postage, yafuoku_bid_num, yafuoku_eval_date, yafuoku_url, yafuoku_image_url, \
            '【取得不可】', '-', '-', '-', '-', '-', '-', '-', \
            '-', \
            '-', \
            '-', \
            '-', \
            '-'
        ])
        
        # 検索ボックスに残っている文字列を消す
        driver.find_element(By.ID, 'twotabsearchtextbox').clear()

# 取得した全データを csv ファイルに出力する
csv_header = [
    'ヤフオク商品名', 'ヤフオク価格(税込)', 'ヤフオク税金', 'ヤフオク送料(税込)', 'ヤフオク入札件数', 'ヤフオク評価日', 'ヤフオク商品URL', 'ヤフオク画像URL', \
    'Amazon商品名', 'Amazon価格(税込)', 'Amazon税金','Amazon送料(税込)' ,'Amazon着日' ,'Amazon在庫' ,'Amazon商品URL' , 'Amazon画像URL' , \
    '商品名前方一致', \
    '画像整合性', \
    'ヤフオク商品名重複', \
    'Amazon商品名重複', \
    '差額'
]
csv_date = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオク＋Amazon調査結果_' + csv_date + '.csv'

with open(csv_file_name, 'w', errors = 'ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_header)
    writer.writerows(yafuoku_amazon_list)
    
print('----- Amazon情報取得完了 -----')